# Ejercicio Práctico: Índice de Representatividad al Senado 2022-2026
El próximo Senado se compone de 50 integrantes, 27 electos en 2021 y 23 en 2017. Con el nuevo sistema proporcional que da importancia a las listas parlamentarias, es interesante evaluar qué integrantes, partidos o pactos son más representativos. Este Índice de Representatividad es una simplificación de un [estudio publicado](https://www.respublica.cl/sections/estudios/) por Instituto Res Publica para la Cámara de Diputados 2017-2022. 

En este ejercicio, consideramos únicamente dos de los tres indicadores que dan lugar al índice real: la cantidad de votos y el porcentaje que obtuvo cada senador en su región. Para ello, utilizaremos dos bases de datos (elecciones 2021 y 2017), las cuales serán preprocesadas y combinadas para luego calcular el Índice de Representatividad simplificado.

Fuentes de datos:
- Resultados Preeliminares Elecciones Senatoriales 2022: [SERVEL](https://www.servel.cl/resultados-preliminares-elecciones-presidenciales-parlamentarias-y-cores-2021/)
- Resultados Senadores 2017: Elaboración propia utilizando [datos históricos del SERVEL](https://www.servel.cl/elecciones-de-senadores-1989-al-2013-por-circunscripcion-electoral/)

*Made with love by Lucas Valenzuela while listening the [best playlist](https://open.spotify.com/playlist/1GVGhANubyLMduxxysZOeS?si=71b2b79db7004ffc)*

# 1. Base de datos 2021
## 1.0. Descargar la base de datos 2021
Partimos descargando los resultados mesa por mesa de cada candidato en la elección de senadores de 2021, en donde se eligieron a 27 de los 50. El archivo `senadores_2021` es un CSV que pesa más de 2GB. Para ello, primero necesitamos instalar gdown.

In [ ]:
!pip install -U --no-cache-dir gdown --pre

# https://drive.google.com/file/d/19qrvGzbfgYJ-vs6AffhUNiNoPJSZQX_N/view?usp=sharing

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=6cc7fd19646d8c64dd57a2358d9982260a74ca4ec46ed400e88bcd4356737c7b
  Stored in directory: /tmp/pip-ephem-wheel-cache-gf5vl0rs/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2


Descargamos el archivo. Este también se encuentra en el siguiente link: `https://drive.google.com/file/d/19qrvGzbfgYJ-vs6AffhUNiNoPJSZQX_N/view?usp=sharing`

In [ ]:
!gdown --id 19qrvGzbfgYJ-vs6AffhUNiNoPJSZQX_N

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=19qrvGzbfgYJ-vs6AffhUNiNoPJSZQX_N
To: /content/senadores_2021.csv
100% 2.07G/2.07G [00:15<00:00, 130MB/s]


**Importante:** `senadores_2017.csv` se encontrará en la sección `Archivos` (botón izquierdo con forma de carpeta). Dicho archivo persistirá allí mientras esté activa el entorno de ejecución. Si no, se eliminará, por lo que es importante descargar o guardar en Drive los archivos que quieran ir guardando.

## 1.1. Importar datos



In [ ]:
import pandas as pd
import numpy as np
import altair as alt # Librería de visualización

Cargamos el achivo csv (se tardará un rato). Otro tipo de formatos que se puede cargar con Excel, JSON y SQL.

In [ ]:
raw = pd.read_csv("/content/senadores_2021.csv", sep=';')
raw.head(2)

,eleccion_id,eleccion_nombre,pais_id,pais_nombre,region_id,region_nombre,provincia_id,provincia_nombre,comuna_id,comuna_nombre,...,sede_colegio,cs_id,cs,distrito_id,distrito,lista,pacto,partido_politico,votos_preliminar_string,votos_provisorio_string
0,5,Elección de Senadores 2021 ...,8056,CHILE ...,3002,DE ANTOFAGASTA ...,4021,ANTOFAGASTA ...,2508,ANTOFAGASTA ...,...,MANUEL ANTONIO MATTA 2176 ANTOFAGASTA ...,5003,CIRCUNSCRIPCIÓN SENATORIAL 3 ...,6003,DISTRITO 3 ...,AA ...,CHILE PODEMOS + ...,RENOVACION NACIONAL ...,8.0,0
1,5,Elección de Senadores 2021 ...,8056,CHILE ...,3002,DE ANTOFAGASTA ...,4021,ANTOFAGASTA ...,2508,ANTOFAGASTA ...,...,MANUEL ANTONIO MATTA 2176 ANTOFAGASTA ...,5003,CIRCUNSCRIPCIÓN SENATORIAL 3 ...,6003,DISTRITO 3 ...,AA ...,CHILE PODEMOS + ...,RENOVACION NACIONAL ...,8.0,0


## 1.2. Examinar y seleccionar columnas
El dataframe contiene más de 1 millón de filas y tiene 38 columnas.

In [ ]:
raw.shape

(1220881, 38)

In [ ]:
raw.eleccion_nombre.value_counts()

Elección de Senadores 2021                            1220881
Name: eleccion_nombre, dtype: int64

In [ ]:
raw.columns

Index(['eleccion_id', 'eleccion_nombre', 'pais_id', 'pais_nombre', 'region_id',
       'region_nombre', 'provincia_id', 'provincia_nombre', 'comuna_id',
       'comuna_nombre', 'circele_id', 'circele_nombre', 'local_id',
       'local_nombre', 'mesa_id', 'mesas_fusionadas', 'mesa_electores',
       'mesa_numero', 'tipomesa', 'mesa_descuadrada_preliminar',
       'mesa_descuadrada_provisorio', 'candidato', 'votos_preliminar',
       'votos_provisorio', 'nvoto', 'electo', 'codigo_colegio',
       'nombre_colegio', 'sede_colegio', 'cs_id', 'cs', 'distrito_id',
       'distrito', 'lista', 'pacto', 'partido_politico',
       'votos_preliminar_string', 'votos_provisorio_string'],
      dtype='object')

Como agregaremos por candidato a senador, solo nos quedaremos con las siguientes columnas:
- `region_nombre`: hay 9 regiones que eligieron senadores.
- `candidato`: hay 177 postulantes al senado.
- `pacto`: hay 12 pactos electorales.
- `partido_politico`: hay 26 partidos políticos.
- `votos_preliminar`: es la cantidad de votos (int) que sacó un candidato en cierta mesa.
- `electo`: funciona como un booleano (0 ó 1) para señalar si dicho candidato fue electo.

In [ ]:
raw = raw[['region_nombre', 'candidato', 'pacto', 'partido_politico', 'votos_preliminar', 'electo']]
raw.head()

,region_nombre,candidato,pacto,partido_politico,votos_preliminar,electo
0,DE ANTOFAGASTA ...,PAULINA ANDREA NUÑEZ URRUTIA ...,CHILE PODEMOS + ...,RENOVACION NACIONAL ...,8,1
1,DE ANTOFAGASTA ...,MARCO ANTONIO DIAZ MUÑOZ ...,CHILE PODEMOS + ...,RENOVACION NACIONAL ...,8,0
2,DE ANTOFAGASTA ...,KATHERINE ELIZABETH LOPEZ RIVERA ...,CHILE PODEMOS + ...,UNION DEMOCRATA INDEPENDIENTE ...,5,0
3,DE ANTOFAGASTA ...,DANIEL GUEVARA CORTES ...,CHILE PODEMOS + ...,UNION DEMOCRATA INDEPENDIENTE ...,0,0
4,DE ANTOFAGASTA ...,PAOLA AMANDA DEBIA GONZALEZ ...,PARTIDO DE LA GENTE ...,PARTIDO DE LA GENTE ...,2,0


In [ ]:
raw.nunique()

region_nombre         9
candidato           177
pacto                12
partido_politico     26
votos_preliminar    321
electo                2
dtype: int64

## 1.3. Filtrar filas
Antes de partir filtrando, guardaremos una copia de seguridad en caso de que nos equivoquemos. 

In [ ]:
# No sirve `raw_copy = raw`, eso solo asigna a otra variable el mismo dataframe
raw_copy = raw.copy()

In [ ]:
raw.electo.value_counts()

0    1086073
1     134808
Name: electo, dtype: int64

Seleccionamos a los candidatos que fueron electos.

In [ ]:
raw = raw[raw.electo == 1]
raw.head()

,region_nombre,candidato,pacto,partido_politico,votos_preliminar,electo
0,DE ANTOFAGASTA ...,PAULINA ANDREA NUÑEZ URRUTIA ...,CHILE PODEMOS + ...,RENOVACION NACIONAL ...,8,1
9,DE ANTOFAGASTA ...,PEDRO ARAYA GUERRERO ...,NUEVO PACTO SOCIAL ...,PARTIDO POR LA DEMOCRACIA ...,23,1
15,DE ANTOFAGASTA ...,ESTEBAN VELASQUEZ NUÑEZ ...,APRUEBO DIGNIDAD ...,FEDERACION REGIONALISTA VERDE SOCIAL ...,15,1
25,DE ANTOFAGASTA ...,PAULINA ANDREA NUÑEZ URRUTIA ...,CHILE PODEMOS + ...,RENOVACION NACIONAL ...,10,1
34,DE ANTOFAGASTA ...,PEDRO ARAYA GUERRERO ...,NUEVO PACTO SOCIAL ...,PARTIDO POR LA DEMOCRACIA ...,24,1


## 1.4. Agrupar filas
Para cada senador, agrupamos los votos que sacó en cada mesa. Subagrupamos por región, pacto y partido político para evitar posible repetición entre dos o más candidatos distintos que se llamen igual.

In [ ]:
raw = raw.groupby(['region_nombre', 'pacto', 'partido_politico', 'candidato']).sum()
raw.head()

votos_preliminar  \
region_nombre                                      pacto                                              partido_politico                                   candidato                                                              
DE ANTOFAGASTA                                 ... APRUEBO DIGNIDAD                               ... FEDERACION REGIONALISTA VERDE SOCIAL               ESTEBAN VELASQUEZ NUÑEZ                        ...             39519   
                                                   CHILE PODEMOS +                                ... RENOVACION NACIONAL                                PAULINA ANDREA NUÑEZ URRUTIA                   ...             15639   
                                                   NUEVO PACTO SOCIAL                             ... PARTIDO POR LA DEMOCRACIA                          PEDRO ARAYA GUERRERO                           ...             24155   
DE COQUIMBO                                    ... APRUEBO DIGNIDAD                               ... PARTIDO COMUNISTA DE CHILE                         DANIEL NUÑEZ ARANCIBIA                         ...             39378   
                                                   CHILE PODEMOS +                                ... UNION DEMOCRATA INDEPENDIENTE                      SERGIO GAHONA SALAZAR                          ...             25715   

                                                                                                                                                                                                             electo  
region_nombre                                      pacto                                              partido_politico                                   candidato                                                   
DE ANTOFAGASTA                                 ... APRUEBO DIGNIDAD                               ... FEDERACION REGIONALISTA VERDE SOCIAL               ESTEBAN VELASQUEZ NUÑEZ                        ...    1437  
                                                   CHILE PODEMOS +                                ... RENOVACION NACIONAL                                PAULINA ANDREA NUÑEZ URRUTIA                   ...    1437  
                                                   NUEVO PACTO SOCIAL                             ... PARTIDO POR LA DEMOCRACIA                          PEDRO ARAYA GUERRERO                           ...    1437  
DE COQUIMBO                                    ... APRUEBO DIGNIDAD                               ... PARTIDO COMUNISTA DE CHILE                         DANIEL NUÑEZ ARANCIBIA                         ...    1865  
                                                   CHILE PODEMOS +                                ... UNION DEMOCRATA INDEPENDIENTE                      SERGIO GAHONA SALAZAR                          ...    1865

In [ ]:
raw.reset_index(inplace=True)
raw.head()

,region_nombre,pacto,partido_politico,candidato,votos_preliminar,electo
0,DE ANTOFAGASTA ...,APRUEBO DIGNIDAD ...,FEDERACION REGIONALISTA VERDE SOCIAL ...,ESTEBAN VELASQUEZ NUÑEZ ...,39519,1437
1,DE ANTOFAGASTA ...,CHILE PODEMOS + ...,RENOVACION NACIONAL ...,PAULINA ANDREA NUÑEZ URRUTIA ...,15639,1437
2,DE ANTOFAGASTA ...,NUEVO PACTO SOCIAL ...,PARTIDO POR LA DEMOCRACIA ...,PEDRO ARAYA GUERRERO ...,24155,1437
3,DE COQUIMBO ...,APRUEBO DIGNIDAD ...,PARTIDO COMUNISTA DE CHILE ...,DANIEL NUÑEZ ARANCIBIA ...,39378,1865
4,DE COQUIMBO ...,CHILE PODEMOS + ...,UNION DEMOCRATA INDEPENDIENTE ...,SERGIO GAHONA SALAZAR ...,25715,1865


Eliminamos la columna que ya no nos sirve.

In [ ]:
raw.drop(columns='electo', inplace=True)

In [ ]:
raw.shape

(27, 5)

In [ ]:
raw.tail()

,region_nombre,pacto,partido_politico,candidato,votos_preliminar
22,METROPOLITANA DE SANTIAGO ...,APRUEBO DIGNIDAD ...,PARTIDO COMUNISTA DE CHILE ...,CLAUDIA PASCUAL GRAU ...,139688
23,METROPOLITANA DE SANTIAGO ...,CHILE PODEMOS + ...,EVOLUCION POLITICA ...,LUCIANO CRUZ-COKE CARVALLO ...,150698
24,METROPOLITANA DE SANTIAGO ...,CHILE PODEMOS + ...,RENOVACION NACIONAL ...,MANUEL JOSE OSSANDON IRARRAZABAL ...,279911
25,METROPOLITANA DE SANTIAGO ...,FRENTE SOCIAL CRISTIANO ...,PARTIDO REPUBLICANO DE CHILE ...,ROJO EDWARDS SILVA ...,249249
26,METROPOLITANA DE SANTIAGO ...,INDEPENDIENTE ...,INDEPENDIENTES ...,FABIOLA CAMPILLAI ROJAS ...,402098


## 1.5. Fusión de dataframes
Nuestro dataframe tiene la suma total de votos de cada diputado. Ahora, queremos obtener el porcentaje que representa para cada uno en su región, por lo que primero tenemos que obtener un segundo dataframe que tenga la cantidad total de votos. Utilizamos la copia.

Eliminamos dos tipos de filas que no son de interés.

In [ ]:
raw_copy.candidato.value_counts()

Total Suma Calculada                                                                                    33445
Total Sufragios Emitidos                                                                                33445
Votos Blancos                                                                                           33445
Votos Nulos                                                                                             33445
VERONICA PILAR PARDO LAGOS                                                                              18166
                                                                                                        ...  
JESUS RODRIGO GUTIERREZ OLIVARES                                                                          517
ALEJANDRO JUAN KUSANOVIC GLUSEVIC                                                                         517
JUAN JOSE CLAUDIO ARCOS SRDANOVIC                                                                         517
JULIO GAST

In [ ]:
totals = raw_copy[(raw_copy.candidato.str.strip() != 'Total Suma Calculada') & 
                  (raw_copy.candidato.str.strip() != 'Total Sufragios Emitidos')]

Creamos el nuevo dataframe con los totales de cada región.

In [ ]:
totals = totals[['region_nombre', 'votos_preliminar']].groupby('region_nombre').sum().reset_index()
totals

,region_nombre,votos_preliminar
0,DE ANTOFAGASTA ...,203622
1,DE COQUIMBO ...,266777
2,DE LOS LAGOS ...,330186
3,DE LOS RIOS ...,161983
4,DE MAGALLANES Y DE LA ANTARTICA CHILENA ...,69912
5,DE ÑUBLE ...,193877
6,DEL BIOBIO ...,619839
7,DEL LIBERTADOR GENERAL BERNARDO O'HIGGINS ...,368343
8,METROPOLITANA DE SANTIAGO ...,2869809


Lo correcto aquí es hacer un merge. Otras operaciones 'pythonisticas' son más costosas. Evitar operaciones como la siguiente:
`````
totals = totals.set_index('region_nombre')
totals_col = []
for r in raw.region_nombre:
  totals_col.append(totals.loc[r].votos_preliminar)
raw['porcentaje'] = totals_col
`````
o lo mismo pero con lista de comprensión
``````
raw['porcentaje'] = [totals.loc[r].votos_preliminar for r in raw.region_nombre]
```````


Este merge agrega a cada senador el total de votos globales de su región a través de la key `region_name`.

In [ ]:
df = pd.merge(raw, totals, on='region_nombre')

In [ ]:
df.head(3)

,region_nombre,pacto,partido_politico,candidato,votos_preliminar_x,votos_preliminar_y
0,DE ANTOFAGASTA ...,APRUEBO DIGNIDAD ...,FEDERACION REGIONALISTA VERDE SOCIAL ...,ESTEBAN VELASQUEZ NUÑEZ ...,39519,203622
1,DE ANTOFAGASTA ...,CHILE PODEMOS + ...,RENOVACION NACIONAL ...,PAULINA ANDREA NUÑEZ URRUTIA ...,15639,203622
2,DE ANTOFAGASTA ...,NUEVO PACTO SOCIAL ...,PARTIDO POR LA DEMOCRACIA ...,PEDRO ARAYA GUERRERO ...,24155,203622


## 1.6. Operaciones con strings y números
Como vemos, las celdas que tienen strings tienen muchos espacios a la derecha que son innecesarios.

In [ ]:
df.iloc[2,0]

'DE ANTOFAGASTA                                                                                      '

In [ ]:
len(df.iloc[2,0])

100

Aparte de eliminar estos espacios innecesarios, queremos solo dejar en mayúsculas la primera letra de cada palabra. Para columnas que contienen string, se debe anteponer `str` para poder realizar acciones propias de strings a cada uno de los elementos de la columna.

In [ ]:
df['candidato'].str.strip()

0               ESTEBAN VELASQUEZ NUÑEZ
1          PAULINA ANDREA NUÑEZ URRUTIA
2                  PEDRO ARAYA GUERRERO
3                DANIEL NUÑEZ ARANCIBIA
4                 SERGIO GAHONA SALAZAR
5                  MATIAS WALKER PRIETO
6          CARLOS IGNACIO KUSCHEL SILVA
7                   IVAN MOREIRA BARROS
8               FIDEL ESPINOZA SANDOVAL
9              MARIA JOSE GATICA BERTIN
10                   IVAN FLORES GARCIA
11           ALFONSO DE URRESTI LONGTON
12    ALEJANDRO JUAN KUSANOVIC GLUSEVIC
13      KARIM ANTONIO BIANCHI RETAMALES
14       GUSTAVO ADOLFO SANHUEZA DUEÑAS
15              LORETO CARVAJAL AMBIADO
16             SEBASTIAN KEITEL BIANCHI
17     ENRIQUE VAN RYSSELBERGHE HERRERA
18              GASTON SAAVEDRA CHANDIA
19          ALEJANDRA SEPULVEDA ORBENES
20                  JAVIER MACAYA DANUS
21            JUAN LUIS CASTRO GONZALEZ
22                 CLAUDIA PASCUAL GRAU
23           LUCIANO CRUZ-COKE CARVALLO
24     MANUEL JOSE OSSANDON IRARRAZABAL


Aplicamos ambos cambios a las columnas no numéricas (object).

In [ ]:
df.dtypes

region_nombre         object
pacto                 object
partido_politico      object
candidato             object
votos_preliminar_x     int64
votos_preliminar_y     int64
dtype: object

In [ ]:
for c in df.dtypes[df.dtypes == object].index.values:
  df[c] = df[c].str.title().str.strip()

In [ ]:
df.head(2)

,region_nombre,pacto,partido_politico,candidato,votos_preliminar_x,votos_preliminar_y
0,De Antofagasta,Apruebo Dignidad,Federacion Regionalista Verde Social,Esteban Velasquez Nuñez,39519,203622
1,De Antofagasta,Chile Podemos +,Renovacion Nacional,Paulina Andrea Nuñez Urrutia,15639,203622


Finalmente, obtenemos el porcentaje de cada senador al dividir sus votos con el total de cada región. A diferencia de operaciones con strings, esta no requiere anteponer algo como `str`, sino que se trata como una numpy series.

In [ ]:
df['porcentaje'] = df.votos_preliminar_x / df.votos_preliminar_y
df.head(2)

,region_nombre,pacto,partido_politico,candidato,votos_preliminar_x,votos_preliminar_y,porcentaje
0,De Antofagasta,Apruebo Dignidad,Federacion Regionalista Verde Social,Esteban Velasquez Nuñez,39519,203622,0.194080
1,De Antofagasta,Chile Podemos +,Renovacion Nacional,Paulina Andrea Nuñez Urrutia,15639,203622,0.076804


## 1.7. Ordenamiento y tratamiento de columnas
Eliminaremos la columna de votos totales que nos fue auxiliar para calcular el porcentaje, renombraremos las columnas y les cambiaremos el orden.

In [ ]:
df.drop(columns='votos_preliminar_y', inplace=True)

In [ ]:
df.columns = ['region', 'pacto', 'partido', 'candidato', 'votos', 'porcentaje']

In [ ]:
df = df.reindex(['candidato', 'region', 'partido', 'pacto', 'votos', 'porcentaje'], axis=1)

Ordenamos el dataframe de mayor a menos según la cantidad de votos.

In [ ]:
df = df.sort_values(by='votos', ascending=False)

Falta
join de las elecciones 2017
calcular el indice de representatividad
ordenar
plotear

## 1.8 Exportar archivo
En caso que quisieramos guardar un dataframe, lo podemos convertir a un formato a conveniencia y descargarlo.

In [ ]:
df.to_csv('clean_senadores_2017.csv')

# 2. Base de datos 2017
## 2.0. Descargar el archivo
El nuevo desafío es unir el dataframe que hemos limpiado con otro que tiene la información de los senadores electos en 2017. Esta información está guardado en un archivo excel. Es un archivo pequeño, dado que ya fue preprocesado y tiene la misma estructura que el dataframe que fuimos armando el el punto anterior.

In [ ]:
!gdown --id 1pnX3F2pKXMTprgPpmtUdaRzT8xRz8rsI

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1pnX3F2pKXMTprgPpmtUdaRzT8xRz8rsI
To: /content/senadores_2017.xlsx
100% 6.74k/6.74k [00:00<00:00, 10.5MB/s]


## 2.1. Importar base de datos
Como podemos apreciar, este dataframe ya está en su estructura deseada, por lo que no hay que preprocesarlo más.

In [ ]:
df2 = pd.read_excel('/content/senadores_2017.xlsx', index_col=0)
df2.head()

,candidato,region,partido,pacto,votos,porcentaje
0,Ordenes Neira Ximena,Aisen Del Gral. Carlos Ibañez,Independiente Por La Democracia,La Fuerza de la Mayoría,5405,0.151409
1,Sandoval Plaza David,Aisen Del Gral. Carlos Ibañez,Union Democrata Independiente,Chile Vamos,7320,0.205054
2,Insulza Salinas Jose Miguel,Arica Y Parinacota,Socialista De Chile,La Fuerza de la Mayoría,14501,0.203050
3,Durana Semir Jose,Arica Y Parinacota,Union Democrata Independiente,Chile Vamos,9639,0.134970
4,Provoste Campillay Yasna,De Atacama,Democrata Cristiano,Convergencia Social,32598,0.342495


In [ ]:
df2.shape

(23, 6)

# 3. Unión de ambos dataframes

Ya con ambos dataframes con el mismo formato, concatenamos ambos por filas y nos quedan los 50 senadores del periodo 2022-2026.

In [ ]:
elected = pd.concat([df, df2], axis=0)
elected

,candidato,region,partido,pacto,votos,porcentaje
26,Fabiola Campillai Rojas,Metropolitana De Santiago,Independientes,Independiente,402098,0.140113
24,Manuel Jose Ossandon Irarrazabal,Metropolitana De Santiago,Renovacion Nacional,Chile Podemos +,279911,0.097536
25,Rojo Edwards Silva,Metropolitana De Santiago,Partido Republicano De Chile,Frente Social Cristiano,249249,0.086852
23,Luciano Cruz-Coke Carvallo,Metropolitana De Santiago,Evolucion Politica,Chile Podemos +,150698,0.052512
22,Claudia Pascual Grau,Metropolitana De Santiago,Partido Comunista De Chile,Apruebo Dignidad,139688,0.048675
19,Alejandra Sepulveda Orbenes,Del Libertador General Bernardo O'Higgins,Federacion Regionalista Verde Social,Apruebo Dignidad,120253,0.326470
16,Sebastian Keitel Bianchi,Del Biobio,Evolucion Politica,Chile Podemos +,64261,0.103674
18,Gaston Saavedra Chandia,Del Biobio,Partido Socialista De Chile,Nuevo Pacto Social,59042,0.095254
8,Fidel Espinoza Sandoval,De Los Lagos,Partido Socialista De Chile,Nuevo Pacto Social,53658,0.162508
21,Juan Luis Castro Gonzalez,Del Libertador General Bernardo O'Higgins,Partido Socialista De Chile,Nuevo Pacto Social,51225,0.139069


## 3.1. Limpieza de datos
Al concatenar, vemos que ambos datasets describen con distintas palabras a un partido político. Además, queremos considerar a independientes que fueron con el cupo de un partido como perteneciente a este. Por último, para los pactos, queremos actualizar a los senadores que fueron electos en 2017 con las coaliciones de sus partidos de 2021.

In [ ]:
elected.partido.value_counts()

Renovacion Nacional                     9
Union Democrata Independiente           9
Evolucion Politica                      4
Partido Socialista De Chile             4
Socialista De Chile                     3
Democrata Cristiano                     3
Independientes                          2
Partido Comunista De Chile              2
Federacion Regionalista Verde Social    2
Partido Por La Democracia               2
Partido Democrata Cristiano             2
Independiente Por La Democracia         2
Por La Democracia                       2
Independiente Renovacion Nacional       2
Partido Republicano De Chile            1
Revolucion Democratica                  1
Name: partido, dtype: int64

In [ ]:
replaces = {'Democrata Cristiano' : 'Partido Democrata Cristiano',
            'Por La Democracia' : 'Partido Por La Democracia',
            'Socialista De Chile' : 'Partido Socialista De Chile',
            'Independiente Por La Democracia' : 'Partido Por La Democracia',
            'Independiente Renovacion Nacional' : 'Renovacion Nacional'}

In [ ]:
elected.pacto.value_counts()

Chile Podemos +            12
Chile Vamos                12
Nuevo Pacto Social          8
La Fuerza de la Mayoría     7
Apruebo Dignidad            4
Convergencia Social         3
Independiente               2
Frente Social Cristiano     1
Frente Amplio               1
Name: pacto, dtype: int64

In [ ]:
pact_replaces = {
    'Chile Vamos' : 'Chile Podemos +',
    'La Fuerza de la Mayoría' : 'Nuevo Pacto Social',
    'Convergencia Social' : 'Nuevo Pacto Social',
    'Frente Amplio' : 'Apruebo Dignidad'
}

replaces.update(pact_replaces)
elected = elected.replace(replaces)

In [ ]:
elected.pacto.value_counts()

Chile Podemos +            24
Nuevo Pacto Social         18
Apruebo Dignidad            5
Independiente               2
Frente Social Cristiano     1
Name: pacto, dtype: int64

## 3.2 Armado del Índice de Representatividad
Para representar al índice, normalizaremos los votos y el porcentaje que obtuvo cada senador para luego promediarlos. Dicha media entre ambos indicadores es el índice de representatividad.

In [ ]:
elected.describe()

,votos,porcentaje
count,50.000000,50.000000
mean,58933.460000,0.145296
std,74157.077263,0.085785
min,4185.000000,0.012392
25%,25824.500000,0.094935
50%,33912.000000,0.131668
75%,57582.000000,0.187493
max,402098.000000,0.445760


In [ ]:
elected['votos_norm'] = (elected.votos - elected.votos.min()) / (elected.votos.max() - elected.votos.min())
elected['porcentaje_norm'] = (elected.porcentaje - elected.porcentaje.min()) / (elected.porcentaje.max() - elected.porcentaje.min())
elected['representatividad'] = (elected.votos_norm + elected.porcentaje_norm) / 2

Ordenamos de mayor a menor e incluimos una columna extra que incluye un ranking discreto.

In [ ]:
elected = elected.sort_values('representatividad', ascending=False)
elected['ranking'] = range(1, 51)
elected.head()

,candidato,region,partido,pacto,votos,porcentaje,votos_norm,porcentaje_norm,representatividad,ranking
26,Fabiola Campillai Rojas,Metropolitana De Santiago,Independientes,Independiente,402098,0.140113,1.000000,0.294717,0.647359,1
13,Karim Antonio Bianchi Retamales,De Magallanes Y De La Antartica Chilena,Independientes,Independiente,31164,0.445760,0.067801,1.000000,0.533901,2
19,Alejandra Sepulveda Orbenes,Del Libertador General Bernardo O'Higgins,Federacion Regionalista Verde Social,Apruebo Dignidad,120253,0.326470,0.291692,0.724737,0.508215,3
24,Manuel Jose Ossandon Irarrazabal,Metropolitana De Santiago,Renovacion Nacional,Chile Podemos +,279911,0.097536,0.692930,0.196471,0.444701,4
17,Chahuan Chahuan Francisco,De Valparaiso,Renovacion Nacional,Chile Podemos +,150065,0.226038,0.366613,0.492988,0.429801,5


## 3.3. Ploteo de la información 
Finalmente, hacemos un par de gráficos para encontrar observaciones interesantes.

In [ ]:
alt.Chart(elected).mark_circle(size=60).encode(
    x='votos',
    y='porcentaje',
    color='pacto',
).interactive()

alt.Chart(...)

In [ ]:
alt.Chart(elected).mark_circle(size=60).encode(
    x='ranking',
    y='representatividad',
    color='pacto',
).interactive()

alt.Chart(...)

In [ ]:
alt.Chart(elected).mark_boxplot(extent='min-max').encode(
    x='representatividad:Q',
    y='pacto:O'
)

alt.Chart(...)

In [ ]:
alt.Chart(elected).mark_boxplot(extent='min-max').encode(
    x='representatividad:Q',
    y='partido:O'
)

alt.Chart(...)

## 3.4. Observaciones
- Independientes tienden a tener altos niveles de representatividad. De hecho, los dos senadores más representativos son Campillai y Bianchi, ambos independientes.
- Apruebo Dignidad es el pacto con más representatividad al observar la distribución de su mediana y su rango intercuartil. 
- Le sigue Chile Podemos +, coalición que a su vez tiene a los dos senadores con menor representatividad (ambos 'arrastrados', Aravena por Kast y Pugh por Chahuán).
- Nuevo Pacto Social tiende a ser la coalición menos representativa.


In [ ]:
elected.head(3)

,candidato,region,partido,pacto,votos,porcentaje,votos_norm,porcentaje_norm,representatividad,ranking
26,Fabiola Campillai Rojas,Metropolitana De Santiago,Independientes,Independiente,402098,0.140113,1.000000,0.294717,0.647359,1
13,Karim Antonio Bianchi Retamales,De Magallanes Y De La Antartica Chilena,Independientes,Independiente,31164,0.445760,0.067801,1.000000,0.533901,2
19,Alejandra Sepulveda Orbenes,Del Libertador General Bernardo O'Higgins,Federacion Regionalista Verde Social,Apruebo Dignidad,120253,0.326470,0.291692,0.724737,0.508215,3


In [ ]:
elected.tail(3)

,candidato,region,partido,pacto,votos,porcentaje,votos_norm,porcentaje_norm,representatividad,ranking
13,Latorre Riveros Juan Ignacio,De Valparaiso,Revolucion Democratica,Apruebo Dignidad,30545,0.046009,0.066246,0.077571,0.071908,48
16,Pugh Olavarria Kenneth,De Valparaiso,Renovacion Nacional,Chile Podemos +,14203,0.021393,0.025176,0.020771,0.022974,49
8,Aravena Acuña Carmen Gloria,De La Araucania,Evolucion Politica,Chile Podemos +,4185,0.012392,0.000000,0.000000,0.000000,50


# 4. Temas no tocados
Por la naturaleza del problema, los siguientes temas no fueron tratados, pero pueden claves para otros casos:
- Tratamiento de NaNs. Métodos relevantes para ello son `isnull`, `dropna` y  `fillna`.
- Operaciones como `pivot_table` y `apply`.
- Combinaciones de dataframes como `append` y `join`.

En este link hay una [cheat sheet](https://www.dataquest.io/blog/pandas-cheat-sheet/) bastante útil que contiene gran parte de los temas tratados en este ejercicio.

Probablemente todas las dudas que tengan pueden ser googleables y sobretodo stackoverflowiables. `Pandas` y `NumPy` son librerías muy poderosas pero necesitan ser usadas con los métodos adecuados para funcionar bien. De otra forma, tendrán códigos que se demoran mucho en correr y que no son eficientes. 

Suerte!